James Gardner and S. Borhanian, 2022 
#### based on quick_start.ipynb by Borhanian 

want to analyse science case/s for CE only:

CE-N 40km with CE-S 40km or 20km

if done, then look at CE-S with one ET detector

In [ ]:
from gwbench import network
from gwbench import wf_class as wfc
from gwbench import detector_response_derivatives as drd
from gwbench import injections

import os, sys
import numpy as np
# from p_tqdm import p_map
from p_tqdm import p_umap
from copy import deepcopy
import matplotlib.pyplot as plt

PI = np.pi

class PassEnterExit:
    def __enter__(self):
        pass

    def __exit__(self, exc_type, exc_val, exc_tb):
        pass

# https://stackoverflow.com/a/45669280; use as ``with HiddenPrints():''
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout
        
def generate_symbolic_derivatives(wf_model_name, wf_other_var_dic, deriv_symbs_string,
                                locs, use_rot, output_path=None):
    """generate symbolic derivatives, from generate_lambdified_functions.py from S. Borhanian 2020
    use network's wf_model_name, wf_other_var_dic, deriv_symbs_string, and use_rot
    will print 'Done.' when finished unless all files already exist in which it will print as such
    
    # # how to print settings as a sanity check
    # print('wf_model_name = \'{}\''.format(wf.wf_model_name))
    # print('wf_other_var_dic = {}'.format(wf.wf_other_var_dic))
    # print('deriv_symbs_string = \'{}\''.format(deriv_symbs_string))
    # print('use_rot = %i'%use_rot)"""
    # skip if derivatives already exist
    file_names = ['par_deriv_WFM_'+wf_model_name+'_VAR_'+deriv_symbs_string.replace(' ', '_')+'_DET_'+key+'.dat' for key in locs]
    file_names.append('par_deriv_WFM_'+wf_model_name+'_VAR_'+deriv_symbs_string.replace(' ra', '').replace(' dec', '').replace(' psi', '').replace(' ', '_')+'_DET_'+'pl_cr'+'.dat')
    path = 'lambdified_functions/'
    file_names_existing = [file_name for file_name in file_names if os.path.isfile(path+file_name)]
    if len(file_names_existing) < len(file_names):
        # if a file doesn't exist, generate them all again
        # waveform
        wf = wfc.Waveform(wf_model_name, wf_other_var_dic)
        # lambidified detector reponses and derivatives
        drd.generate_det_responses_derivs_sym(wf, deriv_symbs_string, locs=locs, use_rot=use_rot,
                                              user_lambdified_functions_path=output_path)   
    else:
        print('All lambdified derivatives already exist.')
        
def basic_network_benchmarking(net, numerical_over_symbolic_derivs=True, only_net=True,
                               numerical_deriv_settings=dict(step=1e-9, method='central', order=2, n=1),
                               hide_prints=True):
    """computes network SNR, measurement errors, and sky area using gwbench FIM analysis
    no return, saves results natively in network (net.snr, net.errs)
    assumes that network is already set up, with waveform set etc."""
    if hide_prints:
        entry_class = HiddenPrints
    else:
        entry_class = PassEnterExit
    with entry_class():
        # compute the WF polarizations and their derivatives
        net.calc_wf_polarizations()
        if numerical_over_symbolic_derivs:
            # --- numerical differentiation ---
            net.calc_wf_polarizations_derivs_num(**numerical_deriv_settings)
        else:
            # --- symbolic differentiation ---
            net.load_wf_polarizations_derivs_sym()
            net.calc_wf_polarizations_derivs_sym()

        # setup antenna patterns, location phase factors, and PSDs
        net.setup_ant_pat_lpf_psds()

        # compute the detector responses and their derivatives
        net.calc_det_responses()
        if numerical_over_symbolic_derivs:       
            # --- numerical differentiation ---
            net.calc_det_responses_derivs_num(**numerical_deriv_settings)
        else:
            # --- symbolic differentiation ---
            net.load_det_responses_derivs_sym()
            net.calc_det_responses_derivs_sym()

        # calculate the network and detector SNRs
        net.calc_snrs(only_net=only_net)
        # calculate the Fisher and covariance matrices, then error estimates
        net.calc_errors(only_net=only_net) #cond_sup=# 1e15 (default) or None (allows all)
        # calculate the 90%-credible sky area (in [deg]^2)
        net.calc_sky_area_90(only_net=only_net)

# https://note.nkmk.me/en/python-numpy-nan-remove/
without_rows_w_nan = lambda xarr : xarr[np.logical_not(np.isnan(xarr)).any(axis=1)]

## Example of simple benchmarking

In [ ]:
# following quick_start.ipynb as a guide 
# Network initialisation

# initialisation is '<config>_<site>'
# other configurations (Tab.4): aLIGO,A+,V+,K+,Voyager-CBO,Voyager-PMO,ET 
# CE configs: <stage>-<arm length>-<optimisation> = CE1/CE2-10/20/30/40-CBO/PMO
# sites/locations (Tab.3): H,L,V,K,I,E,C,N,S; the latter three are Main, North, and South for CE

# choose the desired detectors
# CE-N 40km, CE-S 40km:
# network_spec = ['CE1-40-CBO_C', 'CE1-40-CBO_S']
# network_spec = ['CE2-40-CBO_C', 'CE2-40-CBO_S'] # 2nd stage of development, compare as well
# # CE-N 40km, CE-S 20km:
# network_spec = ['CE1-40-CBO_C', 'CE1-20-PMO_S']
# network_spec = ['CE2-40-CBO_C', 'CE2-20-PMO_S'] # 2nd stage
# locs = ['C', 'S']
network_spec, locs = ['aLIGO_H', 'aLIGO_L', 'aLIGO_V'], ['H', 'L', 'V']

# initialize the network with the desired detectors
net = network.Network(network_spec)

# frequency range
f = np.arange(5., 61.5, 2**-4) # i.e. looking for CBCs around 50 Hz, change for PMO

# choose the desired waveform 
wf_model_name = 'tf2' # TaylorF2, coded in gwbench, allows for symbolic differentiation
wf_other_var_dic = None # for tf2 or tf2_tidal
# other waveforms are available (e.g. tf2_tidal) but require different injection parameters

# pass the chosen waveform to the network for initialization
net.set_wf_vars(wf_model_name=wf_model_name)

# injection parameters
# for GW150914
inj_params = {
    'Mc':    30.9,
    'eta':   0.247,
    'chi1z': 0,
    'chi2z': 0,
    'DL':    475,
    'tc':    0,
    'phic':  0,
    'iota':  PI/4,
    'ra':    PI/4,
    'dec':   PI/4,
    'psi':   PI/4,
    'gmst0': 0}

# assign with respect to which parameters to take derivatives, for the FIM
deriv_symbs_string = 'Mc eta DL tc phic iota ra dec psi'

# assign which parameters to convert to cos or log versions
conv_cos = ('iota', 'dec')
conv_log = ('Mc', 'DL')

# choose whether to take Earth's rotation into account
use_rot = 0
only_net = 1

# pass all these variables to the network
net.set_net_vars(
    f=f, inj_params=inj_params,
    deriv_symbs_string=deriv_symbs_string,
    conv_cos=conv_cos, conv_log=conv_log,
    use_rot=use_rot
    )
print('Network initialised')

In [ ]:
# GW benchmarking
# symbolic derivatives (faster)
generate_symbolic_derivatives(wf_model_name, wf_other_var_dic, deriv_symbs_string, locs, use_rot)

# compute the WF polarizations and their derivatives
net.calc_wf_polarizations()
# --- numerical differentiation ---
# net.calc_wf_polarizations_derivs_num()
# --- symbolic differentiation ---
net.load_wf_polarizations_derivs_sym()
net.calc_wf_polarizations_derivs_sym()

# setup antenna patterns, location phase factors, and PSDs
net.setup_ant_pat_lpf_psds()
# results are accessed like this
# net.detectors[0].Fp # [i] for ith detector in network
# net.detectors[0].Fc # --> not frequency dependent?
# net.detectors[0].Flp
# net.detectors[0].psd # f in future calculations truncated to match psd

# compute the detector responses and their derivatives
# analogous to WF calculation
net.calc_det_responses()
# --- numerical differentiation ---
# net.calc_det_responses_derivs_num()
# --- symbolic differentiation ---
net.load_det_responses_derivs_sym()
net.calc_det_responses_derivs_sym()

# access results (either way) via
# net.detectors[0].hf
# net.detectors[0].del_hf

# calculate the network and detector SNRs
net.calc_snrs(only_net=only_net)
# print(net.snr, net.snr_sq, net.detectors[0].snr, net.detectors[0].snr_sq)

# calculate the network and detector Fisher matrices, condition numbers, ...
# ... covariance matrices, error estimates, and inversion errors
net.calc_errors(only_net=only_net) # finds FIMs, then inverts to find covariance matrix and error estimates of params
# print(net.fisher, net.cond_num, net.cov, net.errs, net.inv_err)
# print(net.detectors[0].fisher, net.detectors[0].cond_num, net.detectors[0].cov,
#       net.detectors[0].errs, net.detectors[0].inv_err)

# calculate the 90%-credible sky area (in [deg]^2)
net.calc_sky_area_90(only_net=only_net)
print('\nBenchmarking complete')

In [ ]:
# print the contents of the detector objects (inside the network)
# net.print_detectors()
# print the contents of the network objects
net.print_network()

# net.get_snrs_errs_cov_fisher_inv_err_for_key(key='network')

# # check if f truncated in any of the detectors to fit the psd
# for i in range(len(net.detectors)):
#     print(np.all(net.f == net.detectors[i].f))

### Replicating Fig 3 from Borhanian 2021 to test understanding, then try for CE only

In [ ]:
# for GW170817 and tf2_tidal

# select network
net, locs = network.Network(['aLIGO_H','aLIGO_L','aLIGO_V']), ['H', 'L', 'V']
# net, locs = network.Network(['CE1-40-CBO_C', 'CE1-40-CBO_S']), ['C', 'S']
# to-do: for CE only, the FIM is ill-conditioned, how to fix this?

# not stated, using one from GW170817 paper
# start with 100 sample points, then move up from there (e.g. to 1e4)
fmin, fmax, fnum = 30, 2048, 100
f = np.linspace(fmin, fmax, fnum)

wf_model_name = 'tf2_tidal'
wf_other_var_dic = None
net.set_wf_vars(wf_model_name=wf_model_name)

# injection parameters for GW170817, reported median values (source-frame)
# using low-spin priors (Borhanian doesn't say which they used)
# https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.119.161101
# subsequent work (2019) has refined these values
base_measured_params = {
    'Mc':    1.188, # Msun
    'eta':   0.2485, # m1=1.48 Msun, m2=1.265 Msun, m2/m1=0.85, Mtot=2.74 Msun, eta=m1*m2/(m1+m2)**2
    'chi1z': 0,
    'chi2z': 0,
    'DL':    40, # MPc
    'tc':    0, # not quoted
    'phic':  0,
#     'iota':  PI/4, # 1000 random instances of these, measured: 55deg
#     'ra':    PI/4, # 1000 random instances of these
#     'dec':   PI/4, # 1000 random instances of these
    'psi':   PI/4, # not quoted, to-do: check effect
    'gmst0': 0, # not quoted but doesn't matter?
    'lam_t': 800, # combined dimensionless tidal deformability
    'delta_lam_t': 0 # not quoted, but approximating as zero because distributed around zero (check)
}

# assign with respect to which parameters to take derivatives, for the FIM, all 12 but not delta_lam_t (or gmst0)
deriv_symbs_string = 'Mc eta chi1z chi2z DL tc phic iota ra dec psi lam_t'

# assign which parameters to convert to log or cos versions for differentiation
conv_log = ('Mc', 'DL', 'lam_t')
conv_cos = ('iota', 'dec')

# choose whether to take Earth's rotation into account
use_rot = 0
# whether to calculate snr, errors, sky area for just the network and not the individual detectors
only_net = 1

# create lambdified derivatives for speed
generate_symbolic_derivatives(wf_model_name, wf_other_var_dic, deriv_symbs_string, locs, use_rot)

# starting with just 10 instances, scale up to 1000 later
num_instances = 1000 # 1000 is O(10 min) with numerical derivatives but is O(1 min) with symbolic derivatives
file_tag = f'{num_instances}instances_{net.label}'

In [ ]:
# angles are sampled to avoid clumping at poles
iota_ra_dec_randoms = np.transpose(injections.angle_sampler(num_instances, np.random.randint(100))[:-1])

def calculate_snr_errs_skyarea(iota_ra_dec):
    """given an array of iota, ra, dec; return an array of the integrate snr, Mc and DL errors, and 90% sky area"""
    iota, ra, dec = iota_ra_dec
    inj_params = dict(**base_measured_params, iota=iota, ra=ra, dec=dec)

    # copy network to avoid parallel operations conflicting, is this an issue when Pool() makes separate instances?
    net_copy = deepcopy(net)

    net_copy.set_net_vars(
        f=f, inj_params=inj_params,
        deriv_symbs_string=deriv_symbs_string,
        conv_cos=conv_cos, conv_log=conv_log,
        use_rot=use_rot
    )

    basic_network_benchmarking(net_copy, numerical_over_symbolic_derivs=False, only_net=only_net)

    if net_copy.wc_fisher: # i.e. net.cond_num < 1e15:
        # net_copy is automatically deleted once out of scope
        return net_copy.snr, net_copy.errs['log_Mc'], net_copy.errs['log_DL'], net_copy.errs['sky_area_90']
    else:
        # try again with different random values
        # to-do: add counter to quantify rate of occurance; seems to be v common with CE only, why?
        #print(f'{net_copy.cond_num:.3g} is ill-conditioned, try again')
        #return calculate_snr_errs_skyarea(np.transpose(injections.angle_sampler(1, np.random.randint(100))[:-1]))
        # alternative to guarantee halting 
        return np.full(4, np.nan)

# array to store integrated SNR, 1sigma error estimates (for Mc and DL), and 90% credible sky area
# must be careful with parallelising that the network is not used simultaneously
# keeping one cpu free to use laptop
results_snr_errs_skyarea = np.array(p_umap(calculate_snr_errs_skyarea, iota_ra_dec_randoms, num_cpus=3))

# save results
np.save(f'data_snr_errs_skyarea/results_snr_errs_skyarea_{file_tag}.npy', results_snr_errs_skyarea)

In [ ]:
# load results
results_snr_errs_skyarea = np.load(f'data_snr_errs_skyarea/results_snr_errs_skyarea_{file_tag}.npy')

# filter out NaNs
results_snr_errs_skyarea = without_rows_w_nan(results_snr_errs_skyarea)

if len(results_snr_errs_skyarea) == 0:
    print('All values are NaN, FIM is ill-conditioned.')
else:
    print('Some values are not NaN.')

In [ ]:
# measured values
# given Appendix E: Asymmetric Systematic Uncertainties, estimating the st.dev. from asymm errs depends on your model
# in the Gaussian case, just average the upper and lower errors that correspond to 68% of the data
# given a 90% credibility interval (x-xlower,x+xupper), the st.dev. is approximated by 1/1.64*1/2(xlower+xupper)
meas_snr = 32.4
# too high eyeballing wrt Borhanian 2021?
meas_Mc_rel_err = 1/1.64*1/2*(0.004+0.002)/1.188 # 1.188+0.004-0.002, range for 90% credibility
# too low eyeballing wrt Borhanian 2021?
meas_LD_rel_err = 1/1.64*1/2*(8+14)/40 # 40+8−14
meas_sky_area = 28
meas_vals = meas_snr, meas_Mc_rel_err, meas_LD_rel_err, meas_sky_area

# plotting
# comparing symmetricised relative errors rather than 90% credible bounds
plt.rcParams.update({'font.size': 14})
fig, axs = plt.subplots(1, 4, figsize=(18, 2), sharey=True, gridspec_kw={'wspace':0.05, 'hspace':0})

for i, ax in enumerate(axs):
    data = results_snr_errs_skyarea[:,i]
    ax.hist(data, histtype='step', facecolor='b', label='tf2_tidal',
            bins=np.geomspace(data.min(), data.max(), 50))
    ax.axvline(meas_vals[i], color='grey')
    ax.set_xscale('log')
    # NB: when there's no major tick, the axis has no reference value
    ax.xaxis.set_minor_locator(plt.LogLocator(base=10.0, subs=0.1*np.arange(1, 10), numticks=10))
    ax.xaxis.set_minor_formatter(plt.NullFormatter())

snr_xlim = axs[0].get_xlim()
snr_threshold_lo = 10 # for detection
snr_threshold_hi = 100 # for high fidelity
for snr_threshold in snr_threshold_lo, snr_threshold_hi:
    efficiency = np.mean(results_snr_errs_skyarea[:,0] > snr_threshold)
    print(f"efficiency of network is {efficiency:.1%} wrt SNR threshold {snr_threshold}")
axs[0].axvspan(snr_xlim[0], snr_threshold_lo, alpha=0.5, color='lightgrey')
axs[0].axvspan(snr_xlim[0], snr_threshold_hi, alpha=0.25, color='lightgrey')
axs[0].set_xlim(snr_xlim)

axs[0].set_ylabel(r'GW170817')
axs[0].set_xlabel(r'integrated SNR, $\rho$')
# error in log(X) is the fractional error in X (i.e. (error in X)/X) by chain rule 
axs[1].set_xlabel(r'chirp mass, $\Delta\mathcal{M}/\mathcal{M}$')
axs[2].set_xlabel(r'luminosity distance, $\Delta D_L/D_L$')
axs[3].set_xlabel(r'sky area, $\Omega$ / $\mathrm{deg}^2$')
axs[0].legend()

fig.align_labels()
fig.savefig(f'GW170817_histograms_{file_tag}.pdf', bbox_inches='tight')
plt.show(fig)
plt.close(fig)
# results for HLV disagree with Borhanian2021: sharpness of first twos' fall-offs, centre of snr distribution, measured Mc and DL, extent of sky area curve
# however, with a difference 1000 instances, the results change --- so maybe just a sampling issue

### Replicating Borhanian and Sathya 2022 injections and detection rates, then for CE only 

In [ ]:
# structure: network, injection loop (inj, benchmark, save data), plot snr histogram, ...
# ... calculate efficiency, calculate detection rate

# try for this network, then CE only (refer to App E for CE discussion), compare to other five in Section 2a?
# --- HLVKI+ ---
network_spec = ['A+_H', 'A+_L', 'V+_V', 'K+_K', 'A+_I']
locs = [x[-1] for x in network_spec]
net = network.Network(network_spec)

# --- BNS ---
# waveform, LAL list: https://lscsoft.docs.ligo.org/lalsuite/lalsimulation/group___l_a_l_sim_inspiral__h.html
wf_model_name = 'lal_bns'
wf_other_var_dic = dict(approximant='IMRPhenomD_NRTidalv2') # for tidal, see https://arxiv.org/abs/1905.06011
# to-do: fix "TypeError: hfpc() missing 2 required positional arguments: 'lam_t' and 'delta_lam_t'"
# --> calculate tidal parameters from sampled m1, m2 in injections.py? requires Love number and radii (i.e. choose an EoS)
# wf_model_name, wf_other_var_dic = 'tf2', None # to-do: stop using this once tidal params found
net.set_wf_vars(wf_model_name=wf_model_name, wf_other_var_dic=wf_other_var_dic)
# injection settings - source
mass_dict = dict(dist='gaussian', mean=1.35, sigma=0.15, mmin=1, mmax=2)
spin_dict = dict(geom='cartesian', dim=1, chi_lo=-0.05, chi_hi=0.05)
# zmin, zmax, seed (use same seeds to replicate results)
redshift_bins = ((0, 0.5, 7669), (0.5, 1, 3103), (1, 2, 4431), (2, 4, 5526), (4, 10, 7035), (10, 50, 2785))
coeff_fisco = 4 # fmax = 4*fisco for BNS, 8*fisco for BBH

base_params = {
    'tc':    0,
    'phic':  0,
#     'gmst0': 0,
    'lam_t': 800, # combined dimensionless tidal deformability, 800 for GW170817
    'delta_lam_t': 0, # assuming zero but can be calculated if m1, m2, Love number, and EoS (i.e. radii) known
}

# derivative settings
# assign with respect to which parameters to take derivatives for the FIM
deriv_symbs_string = 'Mc eta DL tc phic iota ra dec psi'
# assign which parameters to convert to log or cos versions for differentiation
conv_cos = ('dec', 'iota')
conv_log = ('Mc', 'DL', 'lam_t')
numerical_deriv_settings = dict(step=1e-9, method='central', order=2, n=1) # default

# network settings: whether to include Earth's rotation and individual detector calculations
use_rot = 1
only_net = 1

# injection settings - other: number of injections per redshift bin (over 6 bins)
num_injs = 10 # start with 10, then build to 1e6 (how did they compute 1e6 with numerical derivs?)
redshifted = 1 # whether sample masses already redshifted wrt z
if wf_other_var_dic is not None:
    file_tag = f'NET_{net.label}_WF_{wf_model_name}_{wf_other_var_dic["approximant"]}_NUM-INJS_{num_injs}'
else:
    file_tag = f'NET_{net.label}_WF_{wf_model_name}_NUM-INJS_{num_injs}'

In [ ]:
# injection and benchmarking
# concatenate injection data from different bins
inj_data = np.empty((len(redshift_bins)*num_injs, 14))
for i, (zmin, zmax, seed) in enumerate(redshift_bins):
    cosmo_dict = dict(sampler='uniform', zmin=zmin, zmax=zmax)
    # transposed array to get [[Mc0, eta0, ..., z0], [Mc1, eta1, ..., z1], ...]
    # [Mc, eta, chi1x, chi1y, chi1z, chi2x, chi2y, chi2z, DL, iota, ra, dec, psi, z]    
    inj_data[i*num_injs:(i+1)*num_injs] = np.array(injections.injections_CBC_params_redshift(cosmo_dict, mass_dict, spin_dict, redshifted, num_injs=num_injs, seed=seed)).transpose()

def calculate_benchmark_from_injection(inj):
    """given a 14-array of [Mc, eta, chi1x, chi1y, chi1z, chi2x, chi2y, chi2z, DL, iota, ra, dec, psi, z],
    returns a 7-tuple of the
    * redshift z,
    * integrated snr,
    * fractional Mc and DL and absolute eta and iota errors,
    * 90% sky area.
    sigma_log(Mc) = sigma_Mc/Mc is fractional error in Mc and similarly for DL, sigma_eta is absolute,
    while |sigma_cos(iota)| = |sigma_iota*sin(iota)| --> error in iota requires rescaling from output"""
    varied_keys = ['Mc', 'eta', 'chi1x', 'chi1y', 'chi1z', 'chi2x', 'chi2y', 'chi2z', 'DL', 'iota', 'ra', 'dec', 'psi', 'z']
    varied_params = dict(zip(varied_keys, inj))
    z = varied_params.pop('z')
    Mc, eta, iota = varied_params['Mc'], varied_params['eta'], varied_params['iota']
    
    Mtot = Mc/eta**0.6
    #fisco = (6**1.5*PI*Mtot)**-1 # missing some number of Msun, c=1, G=1 factors
    fisco = 4.4/Mtot*1e3 # Hz # from https://arxiv.org/pdf/2011.05145.pdf
    fmin, fmax, df = 5, min(coeff_fisco*fisco, 1024), 1/16 
    f = np.arange(fmin, fmax, df)

    # net_copy is automatically deleted once out of scope (is copying necessary with Pool()?)
    net_copy = deepcopy(net)
    inj_params = dict(**base_params, **varied_params)
    net_copy.set_net_vars(f=f, inj_params=inj_params, deriv_symbs_string=deriv_symbs_string,
                          conv_cos=conv_cos, conv_log=conv_log, use_rot=use_rot)

    basic_network_benchmarking(net_copy, numerical_over_symbolic_derivs=True, only_net=only_net,
                               numerical_deriv_settings=numerical_deriv_settings, hide_prints=True)

    if net_copy.wc_fisher:
        # convert sigma_cos(iota) into sigma_iota
        abs_err_iota = abs(net_copy.errs['cos_iota']/np.sin(iota))
        return (z, net_copy.snr, net_copy.errs['log_Mc'], net_copy.errs['log_DL'], net_copy.errs['eta'],
                abs_err_iota, net_copy.errs['sky_area_90'])
    else:
        # to-do: check if CE only is still ill-conditioned
        return (z, *np.full(6, np.nan))

In [ ]:
calculate_benchmark_from_injection(inj_data[0])
# to-do: fix IndexError: index 0 is out of bounds for axis 0 with size 0 from inside nd.Gradient
# something must be wrong here because the numerical differentiation works normally

In [ ]:
# calculate results: z, snr, errs (logMc, logDL, eta, iota), sky area
results = np.array(p_umap(calculate_benchmark_from_injection, inj_data, num_cpus=os.cpu_count()-1))
# filter out NaNs
results = without_rows_w_nan(results)
if len(results) == 0:
    print('All values are NaN, FIM is ill-conditioned.')
np.save(f'data_B&S2022_replication/results_{file_tag}.npy', results)

In [ ]:
results = np.load(f'data_B&S2022_replication/results_{file_tag}.npy')

In [ ]:
# use integrated SNR rho from standard benchmarking, not sure if B&S2022 use matched filter
snr_threshold_lo = 10 # for detection
snr_threshold_hi = 100 # for high fidelity
for snr_threshold in snr_threshold_lo, snr_threshold_hi:
    efficiency = np.mean(results[:,1] > snr_threshold)
    print(f"average efficiency over all z is {efficiency:.1%} wrt SNR threshold {snr_threshold}")

In [ ]:
# to-do: pass injection data to benchmarking, then calculate a detection rate!
# injections.bns_md_merger_rate(0) # from z=0 outwards
plt.rcParams.update({'font.size': 14})
fig, ax = plt.subplots()
zarr = np.logspace(np.log10(1e-2), np.log10(50), 100)
ax.plot(zarr, list(map(injections.bns_md_merger_rate, zarr)))
ax.set_xscale('log')
ax.set_xlim((1e-2, 50))
ax.set(xlabel='redshift, z', ylabel=r'merger rate / $\mathrm{Gpc}^{-3} \mathrm{yr}^{-1}$') # check units?
plt.show(fig)
plt.close(fig)

In [ ]:
# to-do: fit three-parameter sigmoids to efficiency curves vs redshift

In [ ]:
# to-do: study BBH science case as well

In [ ]:
"""In fact, we see that the three generations (A+, Voyager, and NG) are qualitatively different
with respect to every metric used in this study."""